In [1]:
import time
import numpy as np
import pandas as pd
from tqdm import trange
from sklearn.cluster import AgglomerativeClustering
from pafik.solver import Solver
from pafik.settings import DEFULT_SOLVER

In [17]:

WORKDIR = "."
NUM_POSES = 5_000
N_NEIGHBORS = 5_000
NUM_SOLS = 15_000
LAMBDA = (0.005, 0.05)
STD = 0.25
JOINT_CONFIG_RADS_DISTANCE_THRESHOLD = 2
N_CLUSTERS_THRESHOLD = [10, 15, 20, 25, 30]
SOLUTIONS_SUCCESS_RATE_THRESHOLD_FOR_CLUSTERING_IN_NUM_SOLS = 0.80


solver_param = DEFULT_SOLVER
solver_param.workdir = WORKDIR
solver = Solver(solver_param=solver_param)
num_poses = NUM_POSES
num_sols = NUM_SOLS
n_neighbors = N_NEIGHBORS
verbose = True
batch_size = 5_000
lambda_ = LAMBDA
success_rate_thresold  = SOLUTIONS_SUCCESS_RATE_THRESHOLD_FOR_CLUSTERING_IN_NUM_SOLS

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [3]:
def cluster_based_on_distance(a, dist_thresh=1):
    if len(a) < 3:
        # a minimum of 2 is required by AgglomerativeClustering.
        return 0
    kmeans= AgglomerativeClustering(n_clusters=None, distance_threshold=dist_thresh).fit(a)
    return kmeans.n_clusters_
    # return a[np.sort(np.unique(kmeans.labels_, return_index=True)[1])]
    
def n_solutions_to_reach_n_clusters(J_pose, l2_, ang_):
    """
    # example of useage
    # n_solutions_to_reach_n_clusters(J_hat[i], [10, 15], l2[i], ang[i], 2)
    """
    assert all(n_cluster > 2 for n_cluster in N_CLUSTERS_THRESHOLD)
    
    default_ = -1
    result_n_clusters = np.zeros((len(N_CLUSTERS_THRESHOLD)))
    record = np.full(len(J_pose), default_)
    bound = len(J_pose) - 1
    
    for i, n_cluster in enumerate(N_CLUSTERS_THRESHOLD):
        l, r = 0, bound
        # binary search
        while l < r:
            m = (l + r) // 2
            
            if record[m] == default_:
                J_partial, l2_partial, ang_partial = J_pose[:m], l2_[:m], ang_[:m]
                J_valid = J_partial[(l2_partial < lambda_[0]) & (ang_partial < lambda_[1])]
                record[m] = cluster_based_on_distance(J_valid, JOINT_CONFIG_RADS_DISTANCE_THRESHOLD)
            # print(f"l: {l}, r: {r}, m: {m}, record[m]: {record[m]}")
            if record[m] < n_cluster:
                l = m + 1
            else:
                r = m
        
        # print(f"record: {record}")
        if r == bound:
            result_n_clusters[i] = np.nan
        else:
            result_n_clusters[i] = r
    return result_n_clusters

def n_cluster_analysis(J_hat_, l2_, ang_):
    assert len(J_hat_) == len(l2_) == len(ang_) == NUM_POSES
    return np.array([n_solutions_to_reach_n_clusters(J_hat_[i], l2_[i], ang_[i]) for i in trange(NUM_POSES)])

In [4]:
# P = (NUM_POSES, m)
_, P = solver.robot.sample_joint_angles_and_poses(
    n=num_poses, return_torch=False
)

# F = (NUM_POSES * N_NEIGHBORS, r)
F = solver.F[
                solver.P_knn.kneighbors(
                    np.atleast_2d(P), n_neighbors=n_neighbors, return_distance=False
                ).flatten() 
            ]

P_expand_dim = np.repeat(np.expand_dims(P, axis=1), n_neighbors, axis=1).reshape(-1, P.shape[-1])
F.shape, P_expand_dim.shape

((25000000, 1), (25000000, 7))

In [5]:
begin_time = time.time()
solver.base_std = STD
J_hat = solver.solve_batch(P_expand_dim, F, 1, batch_size=batch_size, verbose=verbose)
l2, ang = solver.evaluate_pose_error_J3d_P2d(J_hat, P_expand_dim, return_all=True)
average_time = (time.time() - begin_time) / num_poses
J_hat = J_hat.reshape(NUM_POSES, N_NEIGHBORS, -1)
l2 = l2.reshape(NUM_POSES, N_NEIGHBORS)
ang = ang.reshape(NUM_POSES, N_NEIGHBORS)

100%|██████████| 5000/5000 [27:09<00:00,  3.07it/s]


In [6]:
df = pd.DataFrame(n_cluster_analysis(J_hat, l2, ang), columns=N_CLUSTERS_THRESHOLD)
df.info(), df.describe()

100%|██████████| 5000/5000 [1:52:52<00:00,  1.35s/it]  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   10      4983 non-null   float64
 1   15      4954 non-null   float64
 2   20      4889 non-null   float64
 3   25      4758 non-null   float64
 4   30      4482 non-null   float64
dtypes: float64(5)
memory usage: 195.4 KB


(None,
                 10           15           20           25           30
 count  4983.000000  4954.000000  4889.000000  4758.000000  4482.000000
 mean    161.826811   452.186112   898.941501  1485.512400  2161.452253
 std     257.505531   458.501561   694.118858   889.213883  1012.427586
 min      15.000000    69.000000   170.000000   280.000000   410.000000
 25%      72.000000   230.000000   494.000000   894.250000  1407.500000
 50%     106.000000   322.000000   687.000000  1224.000000  1917.000000
 75%     160.000000   481.000000  1017.000000  1809.000000  2736.500000
 max    4905.000000  4782.000000  4986.000000  4988.000000  4993.000000)

In [7]:
import torch
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver
from jkinpylib.evaluation import solution_pose_errors

set_seed()
# Build IKFlowSolver and set weights
ik_solver, _ = get_ik_solver("panda__full__lp191_5.25m")
l2_flow = np.zeros((NUM_SOLS, len(P)))
ang_flow = np.zeros((NUM_SOLS, len(P)))
J_flow = torch.empty((NUM_SOLS, len(P), 7), dtype=torch.float32, device="cpu")

begin_time = time.time()
if num_poses < num_sols:
    for i in trange(num_poses):
        J_flow[:, i, :] = ik_solver.solve(
            P[i],
            n=num_sols,
            latent_scale=STD,
            refine_solutions=False,
            return_detailed=False,
        ).cpu()  # type: ignore

        l2_flow[:, i], ang_flow[:, i] = solution_pose_errors(
            ik_solver.robot, J_flow[:, i, :], P[i]
        )
else:
    for i in trange(num_sols):
        J_flow[i] = ik_solver.solve_n_poses(
            P, latent_scale=STD, refine_solutions=False, return_detailed=False
        ).cpu()
        l2_flow[i], ang_flow[i] = solution_pose_errors(ik_solver.robot, J_flow[i], P)
average_time_flow = (time.time() - begin_time) / num_poses

config.py: Using device 'cuda:0'
set_seed() - random int:  541
WorldModel::LoadRobot: /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual

100%|██████████| 5000/5000 [43:05<00:00,  1.93it/s]


In [8]:
df_flow = pd.DataFrame(n_cluster_analysis(J_flow.numpy().transpose((1, 0, 2)), l2_flow.transpose((1, 0)), ang_flow.transpose((1, 0))), columns=N_CLUSTERS_THRESHOLD)
df_flow.info(), df_flow.describe()

100%|██████████| 5000/5000 [26:27:15<00:00, 19.05s/it]   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   10      4947 non-null   float64
 1   15      4904 non-null   float64
 2   20      4803 non-null   float64
 3   25      4631 non-null   float64
 4   30      4277 non-null   float64
dtypes: float64(5)
memory usage: 195.4 KB


(None,
                  10            15            20            25            30
 count   4947.000000   4904.000000   4803.000000   4631.000000   4277.000000
 mean     503.115828   1394.348287   2730.165313   4545.043403   6514.593173
 std     1022.294477   1740.599769   2378.118552   3066.599107   3419.534166
 min       40.000000     89.000000    274.000000    616.000000    975.000000
 25%      180.000000    570.000000   1282.500000   2347.500000   3757.000000
 50%      271.000000    871.000000   1947.000000   3624.000000   5649.000000
 75%      456.000000   1431.250000   3184.000000   5687.000000   8619.000000
 max    14945.000000  14925.000000  14998.000000  14996.000000  14997.000000)

In [25]:
from tabulate import tabulate

class Generate_Diverse_Postures_Info:
    def __init__(self, name, average_time_per_pose, num_poses, num_sols, df, success_rate_threshold):
        # check df counts at least .95 of num_sols
        assert df.count().min() / num_poses > success_rate_threshold, f"df counts at least {success_rate_threshold} of num_sols, {df.count().values / num_poses}"
        self.name = name
        self.success_rate = df.count().values / num_poses
        self.average_time_per_pose = average_time_per_pose
        self.average_time = average_time_per_pose / num_sols
        self.num_sols = num_sols
        self.df = df
    
    # def get_average_time_for_n_clusters(self):
        mean_of_num_solutions_to_reach_n_clusters = self.df.mean().values
        average_time_to_reach_n_clusters = mean_of_num_solutions_to_reach_n_clusters * self.average_time
        # return average_time_to_reach_n_clusters
        self.average_time_to_reach_n_clusters = average_time_to_reach_n_clusters
    
    def __repr__(self):
        return f"Average time per pose: {self.name}: {self.average_time:.6f} s ({self.average_time_per_pose:.2f} s / {self.num_sols} IK solutions)"
        
    def get_list_of_name_and_average_time_to_reach_n_clusters(self):
        return [self.name, *self.average_time_to_reach_n_clusters]
    
    def get_list_of_name_num_sols_and_average_time_to_reach_n_clusters(self):
        return [f"{self.name} ({self.num_sols})", *self.success_rate]
    
P_posture_info = Generate_Diverse_Postures_Info('P', average_time, num_poses, n_neighbors, df, success_rate_thresold)
F_posture_info = Generate_Diverse_Postures_Info('F', average_time_flow, num_poses, num_sols, df_flow, success_rate_thresold)
ratio_average_time_to_reach_n_clusters = P_posture_info.average_time_to_reach_n_clusters / F_posture_info.average_time_to_reach_n_clusters
name_append_ratio = ['ratio', *ratio_average_time_to_reach_n_clusters]

print(tabulate(
    [N_CLUSTERS_THRESHOLD, P_posture_info.get_list_of_name_and_average_time_to_reach_n_clusters(), F_posture_info.get_list_of_name_and_average_time_to_reach_n_clusters(), name_append_ratio],
    headers="firstrow",
))

# print a tbulate of the success rate of the clustering
print(tabulate(
    [N_CLUSTERS_THRESHOLD, P_posture_info.get_list_of_name_num_sols_and_average_time_to_reach_n_clusters(), F_posture_info.get_list_of_name_num_sols_and_average_time_to_reach_n_clusters()],
    headers="firstrow",
))

              10         15         20        25        30
-----  ---------  ---------  ---------  --------  --------
P      0.0123337  0.0344636  0.0685133  0.113219  0.164736
F      0.0173428  0.0480642  0.0941108  0.156671  0.224563
ratio  0.711173   0.717032   0.728007   0.722655  0.733586
               10      15      20      25      30
---------  ------  ------  ------  ------  ------
P (5000)   0.9966  0.9908  0.9778  0.9516  0.8964
F (15000)  0.9894  0.9808  0.9606  0.9262  0.8554


100%|██████████| 3000/3000 [1:12:44<00:00,  1.45s/it]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   10      2988 non-null   float64
 1   15      2966 non-null   float64
 2   20      2940 non-null   float64
 3   25      2839 non-null   float64
 4   30      2672 non-null   float64
dtypes: float64(5)
memory usage: 117.3 KB

(None,
                 10           15           20           25           30
 count  2988.000000  2966.000000  2940.000000  2839.000000  2672.000000
 mean    162.240295   444.310182   920.648980  1492.787249  2181.697605
 std     253.122561   419.647229   725.022072   871.730462  1006.720244
 min      16.000000    69.000000   195.000000   331.000000   533.000000
 25%      73.000000   231.000000   503.000000   914.000000  1426.000000
 50%     106.000000   322.500000   695.000000  1241.000000  1943.000000
 75%     164.000000   485.000000  1044.250000  1820.500000  2755.250000
 max    4774.000000  4862.000000  4996.000000  4997.000000  4991.000000)
 

100%|██████████| 3000/3000 [10:32:47<00:00, 12.66s/it]  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   10      2966 non-null   float64
 1   15      2936 non-null   float64
 2   20      2841 non-null   float64
 3   25      2641 non-null   float64
 4   30      2234 non-null   float64
dtypes: float64(5)
memory usage: 117.3 KB

(None,
                 10          15           20           25           30
 count  2966.000000  2936.00000  2841.000000  2641.000000  2234.000000
 mean    457.335469  1296.43767  2544.591693  4082.816357  5542.084154
 std     734.092405  1319.26164  1822.240410  2241.655126  2321.224216
 min      46.000000   163.00000   399.000000   762.000000  1128.000000
 25%     181.000000   580.75000  1271.000000  2342.000000  3649.250000
 50%     277.000000   886.00000  1955.000000  3547.000000  5250.500000
 75%     450.000000  1454.00000  3148.000000  5235.000000  7402.500000
 max    9944.000000  9987.00000  9980.000000  9982.000000  9998.000000)

Average time per pose: P: 0.37 s / 5000 IK solutions, F: 0.35 s / 10000 IK solutions
Average time per pose: P: 0.37 s / 5000 IK solutions, F: 0.17 s / 5000 IK solutions


           10         15         20        25        30
--  ---------  ---------  ---------  --------  --------
P   0.0120298  0.0329447  0.0682642  0.110687  0.161768
F   0.0145859  0.0401888  0.0745556  0.105584  0.126092